<a href="https://colab.research.google.com/github/NormLorenz/ai-llm-youtube-transcription-utility/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert the YouTube Transaction Utility
## Run in Google Colab in anticipation of using AI to:
* correct noun casing
* correct sentence ending - either a period, a question mark or a exclamation mark
* correct beginning of sentence casing

In [ ]:
# installs

!pip install openai python-dotenv google-generativeai anthropic gradio youtube_transcript_api

In [ ]:
# imports

import os
import io
import sys
import json
import requests
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
from google.colab import userdata
from typing import Dict, List, Union
from youtube_transcript_api import YouTubeTranscriptApi
import re

In [ ]:
# get the utilities module from github

%%bash
wget https://raw.githubusercontent.com/NormLorenz/ai-llm-youtube-transcription-utility/refs/heads/main/utilities.py
wget https://raw.githubusercontent.com/NormLorenz/ai-llm-youtube-transcription-utility/refs/heads/main/README.md

In [ ]:
# add /content to the Python path to allow importing local modules.
# this ensures that Python can find 'utilities.py' as a module.
sys.path.append('/content')

# list content of /content for verification (optional).
display(os.listdir('/content'))

# import utilities.py
import utilities

In [ ]:
# keys

openai_api_key = userdata.get("OPENAI_API_KEY")
claude_api_key = userdata.get("ANTHROPIC_API_KEY")
google_api_key = userdata.get("GOOGLE_API_KEY")
hugging_face_token = userdata.get("HF_TOKEN")

In [ ]:
# initialize

openai = OpenAI(api_key=openai_api_key)
claude = anthropic.Anthropic(api_key=claude_api_key)
google.generativeai.configure(api_key=google_api_key)

OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-haiku-latest"
GOOGLE_MODEL = "gemini-2.5-flash-lite"

In [ ]:
# create a system message

system_message = "You are a Python code assistant. Your task is to analyze Python code and generate high-quality, concise comments and docstrings. Follow these guidelines:"
system_message += "Docstrings: Add a docstring for every function, class, and module. Describe the purpose of the function/class, its parameters, and its return value. Keep the description concise but informative, using proper Python docstring conventions (e.g., Google, NumPy, or reStructuredText format)."
system_message += "Inline Comments: Add inline comments only where necessary to clarify complex logic, important steps, or non-obvious behavior. Avoid commenting on obvious operations like x += 1 unless it involves a nuanced concept. Keep comments short, clear, and relevant."
system_message += "Typing: Strongly type all variables, arguments, classes, functions and modules."
system_message += "General Instructions: Maintain consistency in style and tone. Use technical terminology where appropriate, but ensure clarity for someone with intermediate Python knowledge. Do not over-explain or add redundant comments for self-explanatory code. Follow PEP 257 and PEP 8 standards for style and formatting."

In [ ]:
# create a user prompt

def user_prompt_for(python):
    user_prompt = "Analyze the following Python code and enhance it by adding high-quality, concise docstrings and comments. "
    user_prompt += "Ensure all functions, classes, and modules have appropriate docstrings describing their purpose, parameters, and return values. "
    user_prompt += "Add inline comments only for complex or non-obvious parts of the code. "
    user_prompt += "Follow Python's PEP 257 and PEP 8 standards for documentation and formatting. "
    user_prompt += "Do not modify the code itself; only add annotations.\n\n"
    user_prompt += python
    return user_prompt

In [ ]:
# create a list of dictionaries for the mod

def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [ ]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
def stream_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```python\n','').replace('```','')

In [ ]:
def stream_claude(python):
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[{"role": "user", "content": user_prompt_for(python)}],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace('```python\n','').replace('```','')

In [ ]:
def stream_google(python):
    reply = ""
    gemini = google.generativeai.GenerativeModel(
        model_name=GOOGLE_MODEL,
        system_instruction=system_message
    )
    response = gemini.generate_content(
        user_prompt_for(python),
        stream=True
    )
    for chunk in response:
        if chunk.text:
            reply += chunk.text
            yield reply.replace('```python\n','').replace('```','')

In [ ]:
def optimize(python, model):
    if model=="GPT":
        result = stream_gpt(python)
    elif model=="Claude":
        result = stream_claude(python)
    elif model=="Gemini":
        result = stream_google(python)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [ ]:
def execute_python(code):
        try:
            output = io.StringIO()
            sys.stdout = output
            exec(code)
        finally:
            sys.stdout = sys.__stdout__
        return output.getvalue()

In [ ]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
.table {background-color: red;}
"""

In [ ]:
youTubeURL = "https://www.youtube.com/watch?v=L6HnBjnkKmM"

In [ ]:
# gradio ui

from typing import List, Dict
import gradio as gr

# JavaScript function to copy content from the displayed table
copy_js = """
function() {
    const table = document.querySelector('table');
    const range = document.createRange();
    range.selectNode(table);
    window.getSelection().removeAllRanges();
    window.getSelection().addRange(range);
    document.execCommand('copy');
    alert('Table copied to clipboard!');
    return [];
    }
"""

def clear_fields():
    return [None, ""]

def generate_transcript(url: str) -> List[Dict[str, str]]:
   return [
    {"start": "00:31", "text": "Wie geht's dir?"},
    {"start": "01:23", "text": "Ich trinke Kaffee mit Zucher."},
    {"start": "02:44", "text": "Hallo!"},
   ]

def fetch_transcript(url):
    transcript = generate_transcript(url)
    # transcript = utilities.get_transcript(url)
    html = "<table>\n"
    html += "  <tbody>\n"
    for item in transcript:
        html += f"    <tr><td>{item['start']}</td><td>{item['text']}</td></tr>\n"
    html += "  </tbody>\n"
    html += "</table>"
    return (html)

with gr.Blocks(css=css) as ui:

    gr.Markdown("## YouTube Transcript Utility")
    gr.Markdown("### Creates a HTML table that can be copied and pasted into a Windows OneNote application")

    with gr.Row():
        url = gr.Textbox(label="YouTube Video URL:", value=youTubeURL, lines=1)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="AI Model Name:", value="GPT")
    with gr.Row():
        fetch = gr.Button("Fetch", variant="primary")
        gr.Button("Copy").click(fn=None, inputs=[], outputs=[], js=copy_js)
        clear = gr.Button("Clear")
    with gr.Row():
        html = gr.HTML()

    clear.click(clear_fields, inputs=[], outputs=[url, html])
    fetch.click(fetch_transcript, inputs=[url], outputs=[html])

ui.launch(inbrowser=True, debug=True)
